# Week 3 Assignment

## Victoria Hall
## DSC550
## 12.17.2021

In [1]:
#Importing Packages
import pandas as pd
import numpy as np
from textblob import Word
from textblob import TextBlob
from sklearn.metrics import confusion_matrix

In [2]:
#Reading File
movie_df = pd.read_csv('labeledTrainData.tsv',sep = '\t')
movie_df.head(5)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


### Part 1
#### 2. How many positive and negative reviews?

Sentiment values labeled as 0 have IMBD ratings below 5, indicating a negative review. Sentiment values of 1 indicate postive reviews. Our data is balanced, as 12500 reviews are negative and 12500 is positive. 

In [3]:
#Finding unique values
movie_df['sentiment'].value_counts()

0    12500
1    12500
Name: sentiment, dtype: int64

#### 3 and 4. Check the accuracy of this model. Is this model better than random guessing?

Our model was about 69% accurate. Assuming that with random guessing we would be 50% accurate, our model is more accurate that random guesses.

In [4]:
#Creating function to get polarity
def getPolarity(review):
    return TextBlob(review).sentiment.polarity

#Creating new column for polarity
movie_df["Polarity"] = movie_df['review'].apply(getPolarity)
movie_df.head(5)

#Creating binary classifer for polarity
movie_df["Polarity_Binary"] = np.where(movie_df["Polarity"] >= 0,1,0)
movie_df.head(5)

#Finding Accuracy
movie_df["accuracy"] = movie_df.sentiment - movie_df.Polarity_Binary
movie_df.accuracy.value_counts()
17131/25000

0.68524

### Part 2

#### Converting to lower case, removing special characters, removing stop words, and stemming

In [5]:
#Converting text to lowercase
movie_df["review"] = movie_df["review"].str.lower()
movie_df.head(5)

import unicodedata
import sys


In [6]:
#Removing white space
movie_df["review_strip"] = [string.strip() for string in movie_df["review"]]

#Removing punctuation
#Creating a dictionary of punc.
punctuation = dict.fromkeys(i for i in range(sys.maxunicode)
                           if unicodedata.category(chr(i)).startswith("P"))

movie_df["review_nopunc"] = [string.translate(punctuation) for string in movie_df["review_strip"]]
movie_df.head(5)

,id,sentiment,review,Polarity,Polarity_Binary,accuracy,review_strip,review_nopunc
0,5814_8,1,with all this stuff going down at the moment w...,0.001277,1,0,with all this stuff going down at the moment w...,with all this stuff going down at the moment w...
1,2381_9,1,"\the classic war of the worlds\"" by timothy hi...",0.256349,1,0,"\the classic war of the worlds\"" by timothy hi...",the classic war of the worlds by timothy hines...
2,7759_3,0,the film starts with a manager (nicholas bell)...,-0.053941,0,0,the film starts with a manager (nicholas bell)...,the film starts with a manager nicholas bell g...
3,3630_4,0,it must be assumed that those who praised this...,0.134753,1,-1,it must be assumed that those who praised this...,it must be assumed that those who praised this...
4,9495_8,1,superbly trashy and wondrously unpretentious 8...,-0.024842,0,1,superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...


In [7]:
#Tokenizing. Using apply to tokenize
from nltk.tokenize import word_tokenize
movie_df["tokens"] = movie_df["review_nopunc"].apply(word_tokenize)
movie_df.head(5)

#Removing Stop Words
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")
stop_words = stopwords.words("english")
movie_df["tokens_nostop"] = movie_df["tokens"].apply(lambda x: [word for word in x if word not in stop_words])
movie_df.head(5)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hallt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,sentiment,review,Polarity,Polarity_Binary,accuracy,review_strip,review_nopunc,tokens,tokens_nostop
0,5814_8,1,with all this stuff going down at the moment w...,0.001277,1,0,with all this stuff going down at the moment w...,with all this stuff going down at the moment w...,"[with, all, this, stuff, going, down, at, the,...","[stuff, going, moment, mj, ive, started, liste..."
1,2381_9,1,"\the classic war of the worlds\"" by timothy hi...",0.256349,1,0,"\the classic war of the worlds\"" by timothy hi...",the classic war of the worlds by timothy hines...,"[the, classic, war, of, the, worlds, by, timot...","[classic, war, worlds, timothy, hines, enterta..."
2,7759_3,0,the film starts with a manager (nicholas bell)...,-0.053941,0,0,the film starts with a manager (nicholas bell)...,the film starts with a manager nicholas bell g...,"[the, film, starts, with, a, manager, nicholas...","[film, starts, manager, nicholas, bell, giving..."
3,3630_4,0,it must be assumed that those who praised this...,0.134753,1,-1,it must be assumed that those who praised this...,it must be assumed that those who praised this...,"[it, must, be, assumed, that, those, who, prai...","[must, assumed, praised, film, greatest, filme..."
4,9495_8,1,superbly trashy and wondrously unpretentious 8...,-0.024842,0,1,superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...,"[superbly, trashy, and, wondrously, unpretenti...","[superbly, trashy, wondrously, unpretentious, ..."


In [8]:
#Stemming
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
movie_df["Stems"] = movie_df["tokens_nostop"].apply(lambda x: [porter.stem(word) for word in x])
movie_df.head(5)

,id,sentiment,review,Polarity,Polarity_Binary,accuracy,review_strip,review_nopunc,tokens,tokens_nostop,Stems
0,5814_8,1,with all this stuff going down at the moment w...,0.001277,1,0,with all this stuff going down at the moment w...,with all this stuff going down at the moment w...,"[with, all, this, stuff, going, down, at, the,...","[stuff, going, moment, mj, ive, started, liste...","[stuff, go, moment, mj, ive, start, listen, mu..."
1,2381_9,1,"\the classic war of the worlds\"" by timothy hi...",0.256349,1,0,"\the classic war of the worlds\"" by timothy hi...",the classic war of the worlds by timothy hines...,"[the, classic, war, of, the, worlds, by, timot...","[classic, war, worlds, timothy, hines, enterta...","[classic, war, world, timothi, hine, entertain..."
2,7759_3,0,the film starts with a manager (nicholas bell)...,-0.053941,0,0,the film starts with a manager (nicholas bell)...,the film starts with a manager nicholas bell g...,"[the, film, starts, with, a, manager, nicholas...","[film, starts, manager, nicholas, bell, giving...","[film, start, manag, nichola, bell, give, welc..."
3,3630_4,0,it must be assumed that those who praised this...,0.134753,1,-1,it must be assumed that those who praised this...,it must be assumed that those who praised this...,"[it, must, be, assumed, that, those, who, prai...","[must, assumed, praised, film, greatest, filme...","[must, assum, prais, film, greatest, film, ope..."
4,9495_8,1,superbly trashy and wondrously unpretentious 8...,-0.024842,0,1,superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...,"[superbly, trashy, and, wondrously, unpretenti...","[superbly, trashy, wondrously, unpretentious, ...","[superbl, trashi, wondrous, unpretenti, 80, ex..."


#### Bag of Words Matrix

In [9]:
#importing libraries
from sklearn.feature_extraction.text import CountVectorizer

#Creating the matrix
count= CountVectorizer()
text = np.array(movie_df.Stems.astype(str))
Bag_of_Words = count.fit_transform(text)
Bag_of_Words
#Finding the dimensions.
bagshape = Bag_of_Words.shape
bagshape

(25000, 83041)

#### Term Frequency-Inverse Document Frequency

In [10]:
#importing libraries
from sklearn.feature_extraction.text import TfidfVectorizer

#Creating the matrix
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(text)
feature_matrix

#Finding the dimensions
featshape = feature_matrix.shape
featshape

#Checking if dimensions are the same
print(bagshape==featshape)


True
